In [9]:
%load_ext autoreload
%autoreload 2

In [20]:
DATA_DIR = "data/icra2025-v0" # root dir of your dataset 
""" / 
        ├── cat1
        │   ├── <sensor>_<timestamp>.csv
        │   ├── <sensor>_<timestamp>.pkl
        │   ├── ...
        ├── cat2
        │   ├── <sensor>_<timestamp>.csv
        │   ├── <sensor>_<timestamp>.pkl
        │   ├── ...
        ├── ... 
"""

categories_to_drop = ["empty_disturbed"]

sample_freq_to_encode= 16000 # 16kHz
store_categories_csv = False # create a csv that maps each category to an index, only do this once and make sure it is consistent across all datasets!
N_FOLDS = 5 # set to zero if you do not want to create train/val splits. 
val_ratio = 0.2

In [11]:
# remove all "wav" files from the data directory to avoid inconsistencies
import os
import glob
# glob cannot find files with a leading dot
files = glob.glob(os.path.join(DATA_DIR, "**", "*.wav"), recursive=True)

for f in files:
    os.remove(f)

In [12]:
# find all csv files in the data directory, create dataframe
categories = os.listdir(DATA_DIR)
categories = [c for c in categories if os.path.isdir(os.path.join(DATA_DIR, c))]
print(categories)


import pandas as pd

# create df with 3 columns: filepath, sensor_type, category
entries = []

# for each category, find all csv files
for category in categories:
    print(category)
    csv_files = os.listdir(os.path.join(DATA_DIR, category))
    csv_files = [f for f in csv_files if f.endswith(".csv")]
    print(csv_files)

    for file in csv_files:
        sensor_type = file.split("_")[0]
        filepath = os.path.join(DATA_DIR, category, file)
        # add to df
        entries.append({"filepath": filepath, "sensor_type": sensor_type, "category": category})    

df = pd.DataFrame(entries)

print(len(df))

['3_M6x14', 'empty', 'empty_disturbed']
3_M6x14
['laser_2025-02-18[15].csv', 'mic_2025-02-18[20].csv', 'mic_2025-02-18[11].csv', 'laser_2025-02-18[9].csv', 'laser_2025-02-18[17].csv', 'laser_2025-02-18[14].csv', 'mic_2025-02-18[17].csv', 'mic_2025-02-18[6].csv', 'laser_2025-02-18[16].csv', 'mic_2025-02-18[1].csv', 'laser_2025-02-18[18].csv', 'laser_2025-02-18[8].csv', 'mic_2025-02-18[2].csv', 'laser_2025-02-18[20].csv', 'mic_2025-02-18[5].csv', 'mic_2025-02-18[13].csv', 'laser_2025-02-18[12].csv', 'laser_2025-02-18[5].csv', 'laser_2025-02-18[19].csv', 'mic_2025-02-18[9].csv', 'laser_2025-02-18[3].csv', 'laser_2025-02-18[1].csv', 'mic_2025-02-18[7].csv', 'mic_2025-02-18[15].csv', 'mic_2025-02-18[10].csv', 'mic_2025-02-18[19].csv', 'laser_2025-02-18[6].csv', 'mic_2025-02-18[14].csv', 'mic_2025-02-18[16].csv', 'mic_2025-02-18[4].csv', 'laser_2025-02-18[4].csv', 'mic_2025-02-18[12].csv', 'laser_2025-02-18[10].csv', 'laser_2025-02-18[2].csv', 'laser_2025-02-18[13].csv', 'mic_2025-02-18[18].

In [13]:
# drop categories if needed

for cat in categories_to_drop:
    df = df[df["category"] != cat]
print(f"dataset contains {len(df)} entries")


dataset contains 80 entries


In [14]:
# for each entry, read the csv file save wav file
from converter import SpectrogramCalculator
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

for i, row in tqdm(df.iterrows(), total=len(df)):
 
    filepath = row["filepath"]
    par_dir = os.path.dirname(filepath)
    filename = os.path.basename(filepath)
    filename = filename.replace(".csv", "")


    calc = SpectrogramCalculator(par_dir, filename)
    calc.encode_as_wav(16000) # AST was pretrained on 16kHz audio

    # plot waveform
    # plot_waveform(torch.tensor(calc.data).unsqueeze(0), int(calc.fs))
    # plt.show()

    # save RAM
    del calc
    


100%|██████████| 80/80 [00:40<00:00,  1.96it/s]


In [15]:
if store_categories_csv:
    # create csv with categories
    input("are you sure you want to create a csv with categories? This should only be done once and should be consistent across all datasets. Press enter to continue")

    categories = df["category"].unique()
    categories = {cat: i for i, cat in enumerate(categories)}

    # create csv with entries
    # idx, midname, category_name
    print(categories)

    with open("data/robomic_categories.csv","w") as f:
        f.write("index,mid,display_name\n")
        for name, index in categories.items():
            midname = f"m/robomic{index:02}"
            f.write(f'{index},{midname},"{name}"\n')



categories = open("data/robomic_categories.csv").readlines()
categories = [c.strip().split(",") for c in categories]
categories = categories[1:]
cat_name_to_idx = {c[2].replace('"',''): int(c[0]) for c in categories}
cat_idx_to_midname = {int(c[0]): c[1] for c in categories}
print(f"found {len(cat_name_to_idx)} categories")


found 2 categories


In [16]:
# helper to create json file that defines dataset
import json
def create_json_from_df(df, filename):

    data = []
    for i, row in df.iterrows():
        wav_path = row["filepath"].replace(".csv", ".wav")
        # make relative to filename directory
        par_dir = os.path.dirname(filename)
        wav_path = os.path.relpath(wav_path, par_dir)
        category = row["category"]
        category_id = cat_name_to_idx[category]
        midname_label = cat_idx_to_midname[category_id]
        data.append({"wav": wav_path, "labels": midname_label})
    json_data = {"data": data}
    with open(filename, "w") as f:
        json.dump(json_data, f)



In [17]:
# split by sensor type
laser_df = df[df["sensor_type"] == "laser"]
mic_df = df[df["sensor_type"] == "mic"]

In [18]:
# store entire dataset 
create_json_from_df(laser_df, f"{DATA_DIR}/robomic_all_laser.json")
create_json_from_df(mic_df, f"{DATA_DIR}/robomic_all_mic.json")

In [19]:
import random
random.seed(42)

if N_FOLDS > 0:
    for k in range(N_FOLDS):
        # create train and test splits 
        train_laser_indices = random.sample(range(len(laser_df)), int(len(laser_df) * (1 - val_ratio)))
        val_laser_indices = [i for i in range(len(laser_df)) if i not in train_laser_indices]

        train_mic_indices = random.sample(range(len(mic_df)), int(len(mic_df) * (1 - val_ratio)))
        val_mic_indices = [i for i in range(len(mic_df)) if i not in train_mic_indices]

        train_laser_df = laser_df.iloc[train_laser_indices]
        val_laser_df = laser_df.iloc[val_laser_indices]

        train_mic_df = mic_df.iloc[train_mic_indices]
        val_mic_df = mic_df.iloc[val_mic_indices]

        create_json_from_df(train_laser_df, f"{DATA_DIR}/robomic_train_laser_fold_{k}.json")
        create_json_from_df(val_laser_df, f"{DATA_DIR}/robomic_val_laser_fold_{k}.json")
        create_json_from_df(train_mic_df, f"{DATA_DIR}/robomic_train_mic_fold_{k}.json")
        create_json_from_df(val_mic_df, f"{DATA_DIR}/robomic_val_mic_fold_{k}.json")
